In [ ]:
def scrape_website(objective: str, url: str):
    print('Scraping website...')
    
    headers = {
            'Cache-Control': 'no-cache',
            'Content-Type': 'application/json'
    }
    
    data = {
        "url": url
    }
    
    data_json = json.dumps(data)
    
    post_url = f'https://chrome.browserless.io'
    

In [ ]:
browserless_api_key = '7bd1d93a-e848-4be8-80a5-55297a4f7bf8'
serper_api_key = 'e3ae386de8c45d2b89204dd668b57ed012a72819'

In [ ]:
import json
import requests

In [ ]:
def scrape_website(url: str):
    # scrape website, and also will summarize the content based on objective if the content is too large
    # objective is the original objective & task that user give to the agent, url is the url of the website to be scraped

    print("Scraping website...")
    # Define the headers for the request
    headers = {
        'Cache-Control': 'no-cache',
        'Content-Type': 'application/json',
    }

    # Define the data to be sent in the request
    data = {
        "url": url
    }

    # Convert Python object to JSON string
    data_json = json.dumps(data)

    # Send the POST request
    post_url = f"https://chrome.browserless.io/content?token={browserless_api_key}"
    response = requests.post(post_url, headers=headers, data=data_json)
    
    # Check the response status code
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        for script in soup(["script", "style"]):
            script.decompose()
        text = soup.get_text()
        print("CONTENTTTTTT:", text)

        return text
#         if len(text) > 10000:
#             output = summary(objective, text)
            
#             return output
#         else:
#             return text
    else:
        print(f"HTTP request failed with status code {response.status_code}")

In [ ]:
def search(query):
    url = "https://google.serper.dev/search"
    
    payload = json.dumps({
      "q": query,
      "gl": "us",
      "page": 1
    })

    headers = {
        'X-API-KEY': serper_api_key,
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    print(response.text)

    return response.text

In [ ]:
scrape_website('https://valinonline.com/products/dw-as-503-p12-627')

In [ ]:
search_response = search('CONTRINEX  DW-AS-503-P12-627  Nr. 532157')

print(search_response)

In [ ]:
print(json.dumps(json.loads(search_response), indent=2))

In [ ]:
scrape_website('https://valinonline.com/products/dw-as-503-p12-627')

In [ ]:
import requests

data = {
    "url": "https://example.com/"
}

headers = {
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/json',
}

url = 'https://chrome.browserless.io/content?token=7bd1d93a-e848-4be8-80a5-55297a4f7bf8'
response = requests.post(url, headers=headers, json=data)


if response.status_code == 200:
    print('Request was successful')
    print('Response data:', response.text)
else:
    print('Request failed with status code:', response.status_code)
    print('Response content:', response.text)

In [ ]:
import requests

# Define the API endpoint URL
url = 'https://valinonline.com/products/dw-as-503-p12-627'

# Define headers
headers = {
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/json',
}

# Define the request data in JSON format
data = {
    "url": "https://example.com/"
}

# Make the HTTP POST request
response = requests.post(url, headers=headers, json=data)

# Check the response
if response.status_code == 200:
    print('Request was successful')
    print('Response data:', response.text)
else:
    print('Request failed with status code:', response.status_code)
    print('Response content:', response.text)


In [ ]:
response = requests.post(url, headers=headers, json=data)
response

In [ ]:
from bs4 import BeautifulSoup
import re

soup = BeautifulSoup(response.text, 'html.parser')

sentence_pattern = r'(?<=\.|\?|\!)\s+'
pattern = r'\n+'
# Remove new lines and tabulations before and after each sentence
# cleaned_text = re.sub(sentence_pattern, ' ', text)

# Remove <script> and <style> elements
for script in soup(['script', 'style', 'a']):
    script.extract()

# Select only blocks that have text content
text_blocks = [re.sub(pattern, '\n', block.get_text()) for block in soup.find_all(['p', 'div', 'span']) if block.get_text(strip=True)]

# Print the text content of selected blocks
for block_text in text_blocks:
    if len(block_text) > 0:
        print(block_text)

# print(f'The block text: """{text_blocks[0]}"""')

text_blocks

In [ ]:
with open('manufacturer_detail.txt', 'w') as f:
    f.write('\n'.join(text_blocks))

In [ ]:
soup

In [ ]:
soup.get_text().replace('\n', '  ')

In [ ]:
from langchain.tools import BaseTool
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.schema import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.callbacks import StdOutCallbackHandler


from pydantic import BaseModel, Field
from typing import Type

In [ ]:
class SearchWebsiteInput(BaseModel):
    product_model: str = Field(description = "This is manufacturer product model.")
    part_number: str = Field(description = "This is manufacturer part number of product.")


class SearchWebsiteTool(BaseTool):
    name = "search_price"
    description = "useful when you need to get price about product by their model or part number."
    args_schema: Type[BaseModel] = SearchWebsiteInput
    
    def _run(self, product_model: str, part_number: str):
        return search(f'{product_model} {part_number}')

    def _arun(self, url: str):
        raise NotImplementedError("error here")

In [ ]:
tools = [
    SearchWebsiteTool()
]

In [ ]:
system_message = """

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

system_message = SystemMessage(
    content="You are a world class researcher, who can find price of manufacturer produc by their model, name or part number."
)

In [ ]:
agent_kwargs = {
    "system_message": system_message
}

llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo', openai_api_key='')

memory = ConversationSummaryBufferMemory(memory_key='memory', return_messages=True, llm=llm, max_token_limit=500)

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory,
    callbacks=[
        StdOutCallbackHandler()
    ]
)

In [ ]:

handler = StdOutCallbackHandler()
chain.run(prompt, callbacks=[handler])


In [ ]:
content = agent({"input": """
Hello,

I am looking for a sensor, and I was wondering if it is possible to obtain it from you?

Sensor details:
CONTRINEX DW-AS-503-P12-627
Nr. 532157

And 
    Cable carrier: 1665.030.200.3000-4655 TSO_0 FA_MA Tsubaki 1 piece
    Energy chain: 1665.030.125.140-4189.5 TS0_0 FA_MA Tsubaki 1 piece

Attached is an image of the CONTRINEX M18x1 adapter with M12x1.

Best regards,

Edvard Lesar
Phone: 070 525 775
EUROBOX
Podskrajnik 33, 1380 Cerknica
"""})

content

In [ ]:
import langchain

langchain.debug = False

In [ ]:
content

In [ ]:
print('-------- INPUT --------\n\n')
print(content['input'])
print('\n\n-------- OUTPUT --------\n\n')
print(content['output'])

In [ ]:
# from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [ ]:
pip show langchain


In [ ]:
pip install openai

In [ ]:
pip install tiktoken